eksempel på genkendelse af håndskrift...

inspireret af :

https://medium.com/mlearning-ai/mnist-dataset-of-handwritten-digits-f8cf28edafe


OM Jupyter notebook..

pip install jupyter

derefter fra cmd-consol i pycharm -->jupyter notebook

skift af default browser se : https://www.youtube.com/watch?v=6Ed9KD_fR6U




Der skal installeres torch og matplotlib

In [ ]:
!python.exe -m pip install --upgrade pip

In [ ]:
!pip install torch

In [ ]:
!pip install torchvision

In [ ]:
!pip install matplotlib

importere data  (håndskrevne tal)

In [ ]:
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


# Transform PIL image into a tensor. The values are in the range [0, 1]
t = transforms.ToTensor()

# Load datasets for training and testing.
mnist_training = datasets.MNIST(root='/tmp/mnist', train=True, download=True, transform=t)
mnist_val = datasets.MNIST(root='/tmp/mnist', train=False, download=True, transform =t)




In [ ]:
# Plot some digits.

cols = 8
rows = 2

fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(1.5*cols, 2*rows))
for i, ax in enumerate(axes.flatten()):
    image, label = mnist_training[i]          # returns PIL image with its labels
    ax.set_title(f"Label: {label}")
    ax.imshow(image.squeeze(0), cmap='gray')  # we get a 1x28x28 tensor -> remove first dimension
plt.show()

In [ ]:
mnist_val


In [ ]:
mnist_training[0]

der laves et neuralt netværk.

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(28*28, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)

In [ ]:
print(model)

In [ ]:

# Use Adam as optimizer.
opt = torch.optim.Adam(params=model.parameters(), lr=0.01)


In [ ]:
print(opt)

In [ ]:
# Use CrossEntropyLoss for as loss function.
loss_fn = torch.nn.CrossEntropyLoss()


In [ ]:
# We train the model with batches of 500 examples.
batch_size = 4000  #500
train_loader = torch.utils.data.DataLoader(mnist_training, batch_size=batch_size, shuffle=True)


In [ ]:
type(train_loader)


In [ ]:
losses = []

In [ ]:
for epoch in range(50):  #(10):
    for imgs, labels in train_loader:
        n = len(imgs)
        # Reshape data from [500, 1, 28, 28] to [500, 784] and use the model to make predictions.
        predictions = model(imgs.view(n, -1))
        # Compute the loss.
        loss = loss_fn(predictions, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
        losses.append(float(loss))
    print(f"Epoch: {epoch}, Loss: {float(loss)}")


In [ ]:
plt.plot(losses)

In [ ]:
# Determine the accuracy of our clasifier
# =======================================

# Load all 10000 images from the validation set.
n = 10000
loader = torch.utils.data.DataLoader(mnist_val, batch_size=n)
#images, labels = iter(loader).next()
images, labels = next(iter(loader))

# The tensor images has the shape [10000, 1, 28, 28]. Reshape the tensor to
# [10000, 784] as our model expected a flat vector.
data = images.view(n, -1)

# Use our model to compute the class scores for all images. The result is a
# tensor with shape [10000, 10]. Row i stores the scores for image images[i].
# Column j stores the score for class j.
predictions = model(data)

# For each row determine the column index with the maximum score. This is the
# predicted class.
predicted_classes = torch.argmax(predictions, dim=1)

# Accuracy = number of correctly classified images divided by the total number
# of classified images.
sum(predicted_classes.numpy() == labels.numpy()) / n

In [ ]:
predicted_classes

In [ ]:
cols = 8
rows = 2

fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(1.5*cols, 2*rows))
for i, ax in enumerate(axes.flatten()):
    image, label = images[i],labels[i]          # returns PIL image with its labels
    ax.set_title(f"Label: {label}")
    ax.imshow(image.squeeze(0), cmap='gray')  # we get a 1x28x28 tensor -> remove first dimension
plt.show()

In [ ]:
labels

In [ ]:
errors = []

In [ ]:
print(" i, prediction, label")
for n in range(10000):
    if (predicted_classes[n] != labels[n]):
        print(n,predicted_classes[n],labels[n])
        errors.append(n)
    
print("length:",errors.__len__())

In [ ]:
cols = 8
rows = 2

fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(1.5*cols, 2*rows))
for i, ax in enumerate(axes.flatten()):
    image, label = images[errors[i]],labels[predicted_classes[i]]          # returns PIL image with its labels
    ax.set_title(f"Label: {label}")
    ax.imshow(image.squeeze(0), cmap='gray')  # we get a 1x28x28 tensor -> remove first dimension
plt.show()

In [ ]:
cols = 8
rows = 3

fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(1.5*cols, 2*rows))
for i, ax in enumerate(axes.flatten()):
    image, label = images[errors[i]],labels[predicted_classes[errors[i]]]          # returns PIL image with its labels
    ax.set_title(f"id: {errors[i]}  i: {i} \n  p: {predicted_classes[errors[i]]} l:{labels[errors[i]]}")
    ax.imshow(image.squeeze(0), cmap='gray')  # we get a 1x28x28 tensor -> remove first dimension
plt.show()